In [65]:
import requests

def run_query_subject(entity_idx):
    
    url = "https://query.wikidata.org/sparql"
    query = """

   SELECT (COUNT(*) as ?Triples) WHERE {

   ?subject ?relation wd:<ENTITY>.} 
        """.replace(
            "<ENTITY>", entity_idx)
     
    request = requests.get(url, params={"format": "json", "query": query})
    data = request.json()
    answer = int(data["results"]['bindings'][0]['Triples']['value'])
    return answer

In [66]:
def run_query_object(entity_idx):
    
    url = "https://query.wikidata.org/sparql"
    query = """

   SELECT (COUNT(*) as ?Triples) WHERE {

   wd:<ENTITY> ?relation ?object .} 
        """.replace(
            "<ENTITY>", entity_idx)
     
    request = requests.get(url, params={"format": "json", "query": query})
    data = request.json()
    answer = int(data["results"]['bindings'][0]['Triples']['value'])
    return answer

In [67]:
import pandas as pd
from tqdm import tqdm
import time

In [68]:
data = pd.read_csv("data/adaptive_rag_musique/train.csv")

In [ ]:
for i in tqdm(range(len(data))):

    if pd.notna(data.loc[i, 'bela_base_ents']):
        #and len(data.loc[i, 'bela_base_ents'].split(', ')) != len(ast.literal_eval(data.loc[i, 'bela_ents_graph_subj']))
        cur_ents = data.loc[i, 'bela_base_ents'].split(', ')
        cur_ents_subj = []
        for j in range(len(cur_ents)):
            time.sleep(1)
            cur_ent = cur_ents[j]
            try:
                cur_ents_subj.append(run_query_subject(cur_ent))
            except:
                continue
        
        
        data.loc[i, 'bela_ents_graph_subj'] = str(cur_ents_subj)

 13%|█▎        | 65/500 [02:46<17:54,  2.47s/it]

In [ ]:
for i in tqdm(range(len(data))):

    if pd.notna(data.loc[i, 'bela_base_ents']):
        cur_ents = data.loc[i, 'bela_base_ents'].split(', ')
        cur_ents_obj = []
        for j in range(len(cur_ents)):
            time.sleep(1)
            cur_ent = cur_ents[j]
            try:
                cur_ents_obj.append(run_query_object(cur_ent))
            except:
                continue
        data.loc[i, 'bela_ents_graph_obj'] = str(cur_ents_obj)

 95%|█████████▌| 475/500 [21:43<01:02,  2.48s/it]

In [ ]:
data.to_csv("data/adaptive_rag_musique/train.csv", index = False)